In [49]:
with open("5_4_3reviews.txt") as f:
    reviews = f.read().split("\n")
with open("5_4_3labels.txt") as f:
    labels = f.read().split("\n")
    
reviews_tokens = [review.split() for review in reviews]
print(len(reviews_tokens))
print(len(labels))
print(len(reviews))

123420
123420
123420


In [50]:
from sklearn.preprocessing import MultiLabelBinarizer

onehot_enc = MultiLabelBinarizer()
onehot_enc.fit(reviews_tokens)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews_tokens, labels, test_size=0.4, random_state=None)

split_point = int(len(X_test)/2)
X_valid, y_valid = X_test[split_point:], y_test[split_point:]
X_test, y_test = X_test[:split_point], y_test[:split_point]

In [52]:
import tensorflow as tf
import time

In [53]:
tf.reset_default_graph()

vocab_len = len(onehot_enc.classes_)
inputs_ = tf.placeholder(dtype=tf.float32, shape=[None, vocab_len], name="inputs")
targets_ = tf.placeholder(dtype=tf.float32, shape=[None, 2], name="targets")

h1 = tf.layers.dense(inputs_, 500, activation=tf.nn.relu)
#h2 = tf.layers.dense(h1, 500, activation=tf.nn.relu)
logits = tf.layers.dense(h1, 2, activation=None)
output = tf.nn.sigmoid(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets_))

optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(targets_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [54]:
def label2bool(labels):
    return [[1,0] if label == "positive" else [0,1] for label in labels]

In [55]:
def get_batch(X, y, batch_size):
    for batch_pos in range(0,len(X),batch_size):
        yield X[batch_pos:batch_pos+batch_size], y[batch_pos:batch_pos+batch_size]

In [56]:
epochs = 10
batch_size = 3000

sess = tf.Session()

# Initializing the variables
tic = time.time()
sess.run(tf.global_variables_initializer())
for epoch in range(epochs):
    for X_batch, y_batch in get_batch(onehot_enc.transform(X_train), label2bool(y_train), batch_size):         
        loss_value, _ = sess.run([loss, optimizer], feed_dict={
            inputs_: X_batch,
            targets_: y_batch
        })
        print("Epoch: {} \t Training loss: {}".format(epoch, loss_value))

    acc = sess.run(accuracy, feed_dict={
        inputs_: onehot_enc.transform(X_valid),
        targets_: label2bool(y_valid)
    })

    print("Epoch: {} \t Validation Accuracy: {}".format(epoch, acc))

test_acc = sess.run(accuracy, feed_dict={
    inputs_: onehot_enc.transform(X_test),
    targets_: label2bool(y_test)
})
print("Test Accuracy: {}".format(test_acc))
toc = time.time()
print("Time to completion: " + str(1000*(toc-tic)) + "ms")

Epoch: 0 	 Training loss: 0.692522764206
Epoch: 0 	 Training loss: 0.580520272255
Epoch: 0 	 Training loss: 0.511491715908
Epoch: 0 	 Training loss: 0.475391715765
Epoch: 0 	 Training loss: 0.436409413815
Epoch: 0 	 Training loss: 0.438715726137
Epoch: 0 	 Training loss: 0.428619384766
Epoch: 0 	 Training loss: 0.420236736536
Epoch: 0 	 Training loss: 0.452311128378
Epoch: 0 	 Training loss: 0.416329741478
Epoch: 0 	 Training loss: 0.433669120073
Epoch: 0 	 Training loss: 0.444027274847
Epoch: 0 	 Training loss: 0.389065802097
Epoch: 0 	 Training loss: 0.393432706594
Epoch: 0 	 Training loss: 0.399016797543
Epoch: 0 	 Training loss: 0.378523856401
Epoch: 0 	 Training loss: 0.418591231108
Epoch: 0 	 Training loss: 0.376725673676
Epoch: 0 	 Training loss: 0.371175140142
Epoch: 0 	 Training loss: 0.367582261562
Epoch: 0 	 Training loss: 0.368298172951
Epoch: 0 	 Training loss: 0.356698483229
Epoch: 0 	 Training loss: 0.390983968973
Epoch: 0 	 Training loss: 0.375213772058
Epoch: 0 	 Train